In [1]:
import os
import shutil

directory_out = './file/6.collate'

if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)

# = = = = = = = = = = = = = = =

import pandas as pd

from tqdm import tqdm
import json

# 1. Part

In [2]:
directory_in = './file/2.part'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
for file in list_file:

    print(file)

    # = = = = = = = = = = = = = = =

    df_input = pd.read_excel(f'{directory_in}/{file}',
                             header=0,
                             dtype=str).fillna('')

    df_input['No'] = df_input['No'].astype(int)
    df_input['Packing Unit'] = ['' if packing_unit == '' else int(packing_unit) for packing_unit in df_input['Packing Unit'].tolist()]
    df_input['Quantity Per Packaging Unit'] = ['' if quantity_per_packing_unit == '' else int(quantity_per_packing_unit) for quantity_per_packing_unit in df_input['Quantity Per Packaging Unit'].tolist()]
    df_input['Part Code'] = df_input['Part Code'].astype(int)

    # = = = = = = = = = = = = = = =

    list_series, list_column, list_column_information, list_column_criteria = [], df_input.columns.tolist(), [], []
    for _, series in tqdm(df_input.iterrows(), total=len(df_input), desc='Progress', ncols=77):
        dict_dict, dict_dict_2 = json.loads(series['Json_Information']), json.loads(series['Json_Criteria'])
        dict_, dict_2 = {k: v for dict_temp in dict_dict.values() for k, v in dict_temp.items()}, {k: v for dict_temp in dict_dict_2.values() for k, v in dict_temp.items()}

        for key, value in dict_.items():
            if key in list_column:
                key = f'{key} (Information)'

            if key not in list_column_information:
                list_column_information.append(key)

            series[key] = value

        # = = = = = = = = = = = = = = =

        for key, value in dict_2.items():
            if key in list_column:
                key = f'{key} (Criteria)'

            if key not in list_column_criteria:
                list_column_criteria.append(key)

            series[key] = value

        list_series.append(series)

    # = = = = = = = = = = = = = = =

    df_output = pd.DataFrame(list_series).sort_values(by=['No'],
                                                      ascending=[True],
                                                      ignore_index=True)
    df_output[list_column[:15] + sorted(list_column_information) + sorted(list_column_criteria) + list_column[15:]].to_excel(f'{directory_out}/{file}', index=False)

print()
print('Done ~')

crawler_1.xlsx


Progress: 100%|█████████████████████████████| 10/10 [00:00<00:00, 398.66it/s]


crawler_2.xlsx


Progress: 100%|████████████████████████████| 10/10 [00:00<00:00, 2972.58it/s]


Done ~


# 2. Vehicle

In [3]:
directory_in = './file/5.vehicle'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
for file in list_file:

    print(file)

    # = = = = = = = = = = = = = = =

    df_input = pd.read_excel(f'{directory_in}/{file}',
                             header=0,
                             dtype=str).fillna('')

    df_input['Part Code'] = df_input['Part Code'].astype(int)
    df_input['Link No'] = df_input['Link No'].astype(int)
    df_input['Vehicle No'] = df_input['Vehicle No'].astype(int)

    # = = = = = = = = = = = = = = =

    list_dict, grouped = [], df_input.groupby('Part Code')
    for part_code, group in tqdm(grouped, desc='Progress', ncols=77):
        df_temp = group.reset_index(drop=True)

        dict_vehicle = {}
        for i in range(len(df_temp)):
            make, model, engine, year, year_2 = df_temp.loc[i, 'Make'], df_temp.loc[i, 'Model'], '' if df_temp.loc[i, 'cc'] == '' else f'''{df_temp.loc[i, 'cc']}cc''', int(df_temp.loc[i, 'Year'].split('-')[0][:-2]), 2024 if df_temp.loc[i, 'Year'].split('-')[-1] == '' else int(df_temp.loc[i, 'Year'].split('-')[-1][:-2])

            if make not in dict_vehicle:
                dict_vehicle[make] = {model: {engine: [year] if year == year_2 else [year, year_2]}}
            elif model not in dict_vehicle[make]:
                dict_vehicle[make][model] = {engine: [year] if year == year_2 else [year, year_2]}
            elif engine not in dict_vehicle[make][model]:
                dict_vehicle[make][model][engine] = [year] if year == year_2 else [year, year_2]
            else:
                if year not in dict_vehicle[make][model][engine]:
                    dict_vehicle[make][model][engine].append(year)
                if year_2 not in dict_vehicle[make][model][engine]:
                    dict_vehicle[make][model][engine].append(year_2)

        # = = = = = = = = = = = = = = =

        list_vehicle, list_vehicle_2 = [], []
        for make, dict_model in dict_vehicle.items():
            for model, dict_engine in dict_model.items():
                list_year = [year for list_year in dict_engine.values() for year in list_year]
                year_begin, year_end = min(list_year), max(list_year)
                list_vehicle.append(f'{make} {model} {year_begin}' if year_begin == year_end else f'{make} {model} {year_begin}-{year_end}')

                for engine, list_year in dict_engine.items():
                    year_begin, year_end = min(list_year), max(list_year)
                    list_vehicle_2.append(f'{make} {model} {year_begin} {engine}'.strip() if year_begin == year_end else f'{make} {model} {year_begin}-{year_end} {engine}'.strip())

        # = = = = = = = = = = = = = = =

        list_dict.append({'Part Code': part_code,
                          'Vehicle W/o Engine': '\n'.join(sorted(list_vehicle)),
                          'Vehicle W/ Engine': '\n'.join(sorted(list_vehicle_2))})

    # = = = = = = = = = = = = = = =

    df_output = pd.DataFrame(list_dict).sort_values(by=['Part Code'],
                                                    ascending=[True],
                                                    ignore_index=True)
    df_output.to_excel(f'{directory_out}/{file}', index=False)

print()
print('Done ~')

crawler_1-vehicle.xlsx


Progress: 100%|██████████████████████████████| 10/10 [00:00<00:00, 25.80it/s]


crawler_2-vehicle.xlsx


Progress: 100%|███████████████████████████████| 2/2 [00:00<00:00, 404.60it/s]


Done ~
